# Multi-Site Fire Danger Prediction Using a Spatially Coherent Convolutional Neural Network Model

***Journal of Geophysical Research: Machine Learning and Computation***

**O. Mirones, J. Baño-Medina, S. Brands, J. Bedia**

This notebook reproduces the results presented in the manuscript entitled **Multi-Site Fire Danger Prediction Using a Spatially Coherent Convolutional Neural Network Model**, submitted to the journal *Journal of Geophysical Research: Machine Learning and Computation* by *O. Mirones, J. Baño-Medina, S.Brands, J. Bedia*. 
This paper analyzes the ability of state-of-the-art machine learning algorithms, especially convolutional neural networks, to model the spatial structure of the Fire Weather Index (**FWI**).

## Table of contents:
*  [1 Preparing the R environment](#1-bullet)
*  [2  Load data](#2-bullet)
    *  [2.1  Load predictor data](#2.1-bullet)
    *  [2.2  Load predictand data](#2.2-bullet)
*  [3  Downscaling: Cross-validation](#3-bullet)
    *  [3.1  Benchmark methods](#3.1-bullet)
        *  [3.1.1  Generalized Linear Models](#3.1.1-bullet)
        *  [3.1.2  Convolutional Neural Network Gaussian](#3.1.2-bullet)
    *  [3.2  Convolutional Neural Network Multi-Gaussian](#3.2-bullet)
*  [4  Results](#4-bullet)
    *  [4.1  Correlograms](#4.1-bullet)
    *  [4.2  Mutual Information](#4.2-bullet)
    *  [4.3 Model explainability](#4.3-bullet)

<div class="alert alert-block alert-info">
<b>Note:</b> This notebook was run on a machine with the following technical specifications:
    

- Operating system: Ubuntu 18.04.3 LTS (64 bits)
- Memory: 60 GiB
- Processor: 2x Intel(R) Xeon(R) CPU E5-2670 0 @ 2.60GHz (16 cores, 32 threads)
</div>

## 1. Preparing the R environment <a class="anchor" id="1-bullet"></a>

In particular, the following C4R libraries are used along the notebook:


 * `loadeR` (v1.7.0) for data loading,
 * `transformeR` (v2.1.0) for data manipulation, 
 * [`downscaleR`](https://doi.org/10.5194/gmd-13-1711-2020) (v3.3.2) for downscaling and
 * [`downscaleR.keras`](https://doi.org/10.5194/gmd-13-2109-2020) (v1.0.0) for downscaling with neural networks and

A frozen version of the above libraries and all the ones needed to reproduce the manuscript are installable through the `environment.yml` file using the `mamba` package, by executing the following commands in your command shell terminal: 
```shell
$ mamba env create -n deep-fwi --file=environment.yml
```

Once installed, you activate the newly created `deep-fwi` environment using `conda` by typing the following command:
```shell
$ conda activate deep-fwi
```

Once in the environment, type `R` and load the libraries:

In [ ]:
### Loading libraries
library(loadeR)
library(transformeR)
library(downscaleR)
library(magrittr)
library(downscaleR.keras)
library(tfprobability)
library(MASS)
library(VALUE)
library(geosphere)
library(scales)
library(abind)
library(grid)
library(gridExtra)
library(lattice)

We load some auxiliary functions from the `utils` directory. The file `./utils/topologiesCNN.R` contains the code implementing the convolutional neural networks used in this study, which are based on tensorflow-keras_

+ `./utils/downscaleCV.keras.tfprobability.R` is a function enabling cross-validation over a convolutional neural network coded with the library tensorflow-probability. 
+ `./utils/dimFix.R` completes missing dimensions of VALUE objects. 
+ `./utils/corrMat.VALUE.R`computes the cross correlation matrices between stations that serve as input for plotting functions. `dimFix.R` and `corrMat.VALUE.R` files are a variation of its namesake from the package `VALUE`, adapted for use with notebook data structures. 
+ `./utils/corrL_allST.R` returns the correlation length for the `corrMat.VALUE` output. The `corrL_allST` function uses the file `./utils/corr_length.R` to calculate the correlation length based on the smoothed fit to the given data.
+ `./utils/smooth_corr.R` smooths using a loess filter with standard settings.
+ `./utils/miMat.VALUE.R`computes the mutual information between stations. This file is a version of ist namesake from the package `VALUE`, adapted for the Fire Weather Index (`FWI`). 
+ `./utils/ascii_export.R`, is designed to save the predictions obtained in ASCII format, ensuring compatibility with various text editors and data processing tools.
+  `./utils/station_table.csv` contains basic characteristics of the stations studied in this work. This information is particularly useful for plotting the explainable saliency maps for each climatic region.

In [ ]:
source("./utils/topologiesCNN.R")
source("./utils/downscaleCV.keras.tfprobability.R")
source("./utils/dimFix.R")
source("./utils/corrMat.VALUE.R")
source("./utils/corrL_allST.R")
source("./utils/corr_length.R")
source("./utils/smooth_corr.R")
source("./utils/miMat.VALUE.R")
source("./utils//ascii_export.R")

Below we define parameters related to the cross-validation procedure. 

In [ ]:
folds <- list(1985:1991, 1992:1998, 1999:2004, 2005:2011)
vars <- c("hus850", "ta850", "tas", "ua850", "va850")

## 2 Load data <a class="anchor" id="2-bullet"></a>
In this section we illustrate how to load the data into our `R` environment. Previously, we need to download the [data from the Zenodo repository](https://doi.org/10.5281/zenodo.8381438) and store it in our working directory. For the notebook to work succesfully, it is important to preserve the structure of directories encountered in Zenodo. In addition we create a new folder where we will store the estimated fields named `./data/predictions/`. The folder `figures` is created to store the figures of the results obtained. 

**Note**: The user must keep the directory structure of the downloaded data for the correct reproducibiliy of the code.

In [ ]:
destfile <- tempfile()
download.file(url = "https://zenodo.org/records/8381438/files/data.zip?download=1",
              destfile = destfile)
unzip(destfile, exdir = ".")

In [ ]:
### Store in "./data/predictions" the output data for the CNN models and predictand
### and the figures produced in section 4 in "./figures/" .
if (!dir.exists("./data/predictions/")) dir.create("./data/predictions/", showWarnings = FALSE)
###
if (!dir.exists("./figures/")) dir.create("./figures/", showWarnings = FALSE)

## 2.1 Load predictor data <a class="anchor" id="2.1-bullet"></a>
We load the following predictor variables: specific humidity at 850hPa (`hus850`), air temperature at 850 hPa (`ta850`), air surface temperature (`tas`), zonal wind velocity at 850 hPa (`ua850`), and meridional wind velocity at 850 hPa (`va850`). These variables are stored as `.nc` files in the directory `./data/predictors/`. We use `loadGridData` from library `loadeR` to load the data into our `R` session. Then we call function `makeMultiGrid` to bind these predictor variables into a single object named `x`.

In [ ]:
### Loading predictor data
x <- lapply(vars, FUN = function(var) {
  out <- loadGridData(paste0("./data/predictors/", var, ".nc"), var = var)
  print(sprintf("Variable: %s loaded!", var))
  return(out)
}) %>% makeMultiGrid()

## 2.2 Load predictand data <a class="anchor" id="2.1-bullet"></a>

In [ ]:
### Loading predictand data from Zotero database
ds <- "./data/predictands"
di <- dataInventory(ds)
str(di)

y <- loadStationData(dataset = ds,
                       var = "fwi13",
                       lonLim = c(0,15),
                       latLim = c(35,40),
                       years = 1985:2011,
                       tz = "UTC")

## 3 Downscaling: Cross-validation <a class="anchor" id="3-bullet"></a>
In this section we perform cross-validation over the machine learning methods used in this study: analogs, generalized linear models, and three different convolutional neural network architectures. Therefore, we split the data into the following 4 chronological folds: 1985-1991, 1992-1998, 1999-2004, 2005-2011. We store the resulting predictions into the previously created `./data/predictions/` folder. These files will be used in the next section. 

In [ ]:
pred.dir <- "./data/predictions/"

The results presented in this section are predominantly based on libraries `downscaleR`, `transformeR` and `downscaleR.keras`. In particular we use the function `downscaleCV` from library `downscaleR` to perform cross-validation for the analogs and generalized linear models. Similary, we use the function `downscaleCV.keras` from `downscaleR.keras` to cross-validate the convolutional neural networks. The only exception is the convolutional model multi-site multi-gaussian which builds on function `downscaleCV.keras.tfprobability` located in the `utils` directory.

The predictions are stored in an ASCII text file through the utilization of the `./utils/ascii_export.R` script. This function takes several inputs, including the predictions themselves, denoted as `pred`, the desired file name, represented by `filename`, and the directory where the predictions are to be stored, indicated as `dest.dir`. Additionally, it requires the specification of reference stations, which should correspond to the information found in the `./data/predictands/ascii_dataset/stations.txt` file.

## 3.1 Benchmark methods <a class="anchor" id="3.1-bullet"></a>

## 3.1.1 Generalized Linear Model <a class="anchor" id="3.1.1-bullet"></a>

In [ ]:
### Downscaling generalized linear model
number_of_neighbours = 16
p <- downscaleCV(x = x,
                 y = y,
                 method = "GLM",
                 family = "gaussian",
                 prepareData.args = list("local.predictors" = list(n = number_of_neighbours, vars = getVarNames(x))),
                 scaleGrid.args = list(type = "standardize"),
                 folds = folds)

ascii_export(pred = p, filename = "FWI-GLM", 
             stations = "./data/predictands/stations.txt", 
             dest.dir = pred.dir)

## 3.1.2 Convolutional Neural Network Gaussian <a class="anchor" id="3.1.2-bullet"></a>

In [ ]:
### Downscaling convolutional neural network multi-site gaussian
model <- cnn_model(topology = "cnn-gaussian",
                   input_shape = c(getShape(x, "lat"), getShape(x, "lon"), getShape(x, "var")),
                   output_shape = getShape(y, "loc"),
                   kernel_size = c(3,3),
                   neurons = c(50, 50))
p <- downscaleCV.keras(x = x,
                       y = y,
                       model = model,
                       loss = "gaussianLoss",
                       prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                       compile.args = list("loss" = gaussianLoss(last.connection = "dense"), "optimizer" = optimizer_adam(lr = 0.0001)),
                       fit.args = list("batch_size" = 100, "epochs" = 10000, "validation_split" = 0.1, "verbose" = 0, "callbacks" = list(callback_early_stopping(patience = 30))),
                       scaleGrid.args = list(type = "standardize"),
                       folds = folds)

p <- computeTemperature(mean = subsetGrid(p, var = "mean"), log_var = subsetGrid(p, var = "log_var"))

ascii_export(pred = p, filename = "FWI-CNN-G", 
             stations = "./data/predictands/stations.txt", 
             dest.dir = pred.dir)

## 3.2 Convolutional Neural Network Multi-Gaussian <a class="anchor" id="3.2-bullet"></a>

In [ ]:
### Downscaling convolutional neural network multi-site multi-gaussian
negloglik <- function (x, rv_x) - (rv_x %>% tfd_log_prob(x))
p <- downscaleCV.keras.tfprobability(x = x,
                                     y = y,
                                     samples = 1,
                                     cnn_model.args = list(topology = "cnn-multi-gaussian",
                                                           input_shape = c(getShape(x, "lat"), getShape(x, "lon"), getShape(x, "var")),
                                                           output_shape = getShape(y, "loc"),
                                                           kernel_size = c(3,3),
                                                           neurons = c(200, 200)),
                                     type_nn = "multivariate-gaussian",
                                     prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                                     compile.args = list("loss" = negloglik, "optimizer" = optimizer_adam(lr = 0.0001)),
                                     fit.args = list("batch_size" = 100L, "epochs" = 10000L, "validation_split" = 0.1, "verbose" = 0L, "callbacks" = list(callback_early_stopping(patience = 30))),
                                     scaleGrid.args = list(type = "standardize"),
                                     folds = folds)

ascii_export(pred = p, filename = "FWI-CNN-MG", 
             stations = "./data/predictands/stations.txt", 
             dest.dir = pred.dir)

## 4 Results <a class="anchor" id="4-bullet"></a>

In this section, we will present an analysis of the results obtained from the different predictions. Firstly, we will display the correlograms that have been constructed for each prediction. Secondly, we will present the mutual information matrices. Lastly, we show the saliency maps, spatial representations of the feature importance in the model outputs.

## 4.1 Correlograms <a class="anchor" id="4.1-bullet"></a>

Here, we assess the strength of linear relationships between observed and predicted `FWI` time series among different locations. We accomplish this by calculating pairwise *Spearman's* cross-correlations among all pairs of stations. To visualize these relationships, we construct a location plot by plotting the cross-correlation value for each station pair against their respective geographical distances.

Next, we fit a local 2nd-order polynomial (loess) to obtain two curves—one for predictions and one for observations. This enables a straightforward visual assessment of the spatial correlation structure of predictions compared to the reference observations.

For the analysis, we define the fire season as June to September (JJAS), and we set a threshold ($0.4$) for the correlation length and specify the correlation type. The correlations are then computed using the `corrL_allST` function.

Additionally, we calculate the Root Mean Square Error (`RMSE`), Mean Absolute Error (`MAE`), and the correlation length bias (`CL bias`). These metrics provide valuable insights into the performance of the predictions compared to the reference observations. All the computed values, including correlations, correlation length, `RMSE`, `MAE`, and `CL bias`, are included in the correlation plots.

To facilitate easy access and reference, the resulting figure will be stored in the `figures` directory.

In [ ]:
###Defining season, threshold and correlation type

sea <- "JJAS"
tr <- .4
corrtype <- "spearman"

In [ ]:
###Computing and plotting the correlogram for the reference observations
pdf("./figures/correlograms.pdf", width = 14, height = 8.5)
par(mfrow = c(2,3))

corrL_obs <- corrL_allST(data=y, type = "after", corrtype = corrtype,tr=tr,season=sea)

plot(corrL_obs$JJAS, col = alpha('grey', 0.3), pch = 16, ylim = c(-0.3,1),
     xlab = "Distance [km]", ylab = "Correlation", main = "Obs", cex.lab = 1.5, cex.main = 1.5, cex.axis = 1.5,
     las = 1)
abline(h = tr, lty = 2)
abline(v = corrL_obs[["JJA"]][["corrL"]], lwd = 2, col = 'grey')
lines(corrL_obs[["JJAS"]][["x_fit"]], corrL_obs[["JJAS"]][["y_fit"]], col = 'grey', lwd = 2.5)
grid()

In [ ]:
###Computing and plotting the correlograms for the predictions
preds <- c("FWI-GLM", "FWI-CNN-G", "FWI-CNN-MG")


for (i in c(1:length(preds))) {
  
  ###Loading prediction and computing the correlation length
  
  p <- loadStationData("./data/predictions/",var = preds[i])
  corrL_pred <- corrL_allST(data=p, type = "after", corrtype = corrtype,tr=tr,season=sea)
  
  ###Computing RMSE and MAE
  
  d <- corrL_obs[["JJAS"]][["y"]] - corrL_pred[["JJAS"]][["y"]]
  mae <- mean(abs(d))
  mse <- mean((d)^2)
  rmse <- sqrt(mse)
  bias.corrL <- corrL_obs[["JJAS"]][["corrL"]]-corrL_pred[["JJAS"]][["corrL"]]
  
  ###Visualization
  plot(corrL_pred$JJAS, col = alpha('red', 0.3), pch = 16, ylim = c(-0.3,1),
       xlab = "Distance [km]", ylab = "Correlation",
       main = sub("FWI-", "", preds[i]), 
       cex.lab = 1.5, cex.main = 1.5, cex.axis = 1.5, las = 1)
  abline(h = tr, lty = 2)
  abline(v = corrL_obs[["JJAS"]][["corrL"]], lwd = 2, col = 'grey')
    
  lines(corrL_obs[["JJAS"]][["x_fit"]], corrL_obs[["JJAS"]][["y_fit"]], col = 'grey', lwd = 2.5) 
  abline(v = corrL_pred[["JJAS"]][["corrL"]], lwd = 2, col = 'red')
  lines(corrL_pred[["JJAS"]][["x_fit"]], corrL_pred[["JJAS"]][["y_fit"]], col = 'red', lwd = 2.5)
    
  text(1225,0.85, paste0("RMSE: ", round(rmse,3)), cex = 1.45)
  text(1225,0.75, paste0("MAE: ", round(mae,3)), cex = 1.45)
  text(1225,0.95, paste0("CL bias: ", round(bias.corrL, 3)), cex = 1.45)
  grid()
 
}

dev.off()

## 4.2 Mutual Information <a class="anchor" id="4.2-bullet"></a>

We proceed to plot each mutual information (`MI`) value, denoted as $M_{i,j}$, against the distance between the corresponding locations, i.e., $i$ and $j$. To capture the underlying patterns, we fit a second-degree loess curve to these plots.

To determine the mutual information lengths (`MIL`), we establish MI thresholds for both the observations and the different downscaling methods. In this analysis, we utilize a MI threshold of $0.05$. This threshold allows for comparable results to correlation length analyses and aids in the identification of potential new information regarding the performance of each method.

Based on the climatic regions, the stations are grouped together in a matrix. This matrix exhibits three primary groupings, each with a quadrilateral shape. These groupings correspond to the following climatic regions: the *Atlantic* region, the *Continental Mediterranean* region, and the *Coastal Mediterranean* stations.

In [ ]:
###Defining the order of the stations

y[["Metadata"]][["name"]] <- c("REUS","S.COMPOSTELA","VIGO","SORIA","VALLADOLID","ZAMORA","LEON","SALAMANCA",
                               "MADRID-BARAJAS","MADRID-RETIRO","CIUDAD REAL","BADAJOZ","GRANADA",
                               "SEVILLA","MORON","JEREZ","ALMERIA","MURCIA","ALICANTE-ALTET",
                               "ALICANTE","CUENCA","VALENCIA-AER.","VALENCIA","LOGRONO","DAROCA","TORTOSA",
                               "MALLORCA","MENORCA","IBIZA")

###Defining the FWI90 event for the mutual information

prob <- .9

In [ ]:
###defining the color palettes
mi.colors <- RColorBrewer::brewer.pal(11, "Reds") %>% colorRampPalette()
spec.colors <- RColorBrewer::brewer.pal(11, "Spectral") %>% colorRampPalette()

In [ ]:
#modification of stations order
station.ids <- y[["Metadata"]][["station_id"]][c(2:10,21,24,13,12,11,25,1,27:29,18:20,22,23,26,14:17)]
y <- subsetGrid(y, station.id = station.ids) %>% redim(member = FALSE, loc = TRUE)

In this step, we compute the `MI` matrix for both the reference observations and the predictions. To perform these computations, we utilize the `miMat.VALUE` function. This function enables us to calculate the `MI` values and construct the `MI` matrix, which represents the pairwise `MI` between different locations.

In [ ]:
### Mutual Information matrix computation for the observations

mi.matrix.obs <- miMat.VALUE(y,
                         predictionObj = NULL,
                         season = "JJAS",
                         threshold = NULL,
                         prob = prob)

In [ ]:
### Mutual Information matrix computation for the predictions
mi.matrix.list <- list()
preds.names <- c("GLM","CNN-G", "CNN-MG")

for (i in c(1:length(preds))) {

  ### Loading predictions
    
  p <- loadStationData("./data/predictions/",var = preds[i])
  
  ###Ordering stations by location
  
  p <- subsetGrid(p, station.id = station.ids) %>% redim(member = FALSE, loc = TRUE)
  
  ###Computing the Mutual Information matrix
  
  mi.matrix.list[[i]] <- miMat.VALUE(p,
                predictionObj = NULL,
                season = "JJAS",
                threshold = NULL,
                prob = prob)
  
  
}

Lastly, we proceed to plot the `MI` matrices for the `FWI90` event during the fire season (June to September). The first matrix computed represents the MI for the reference observations, displayed in the upper triangle, and the best-performing model (`CNN-MG`) in the lower triangle. The bottom left and right matrices depict the `MI` bias of the methods compared to the observations.

In the `MI` bias matrices, the analogs method is excluded from the representation since `MI` biases are negligible by construction. These matrices are divided to show the bias corresponding to the different methods in the upper and lower triangles. Only pairs of stations with `MI` values $\ge 0.05$ for the observations are displayed in the `MI` bias matrices.

In [ ]:
###Setting some atributes for the figure
scales.list <- list(x = list(labels = attributes(mi.matrix.obs[[1]])$station_names, rot = 90,
                             at = seq(1,ncol(mi.matrix.obs[[1]]),1), cex = .65),
                    y = list(labels = attributes(mi.matrix.obs[[1]])$station_names,
                             at = seq(1,ncol(mi.matrix.obs[[1]]),1), cex = .65),
                   alternating = 2)

In [ ]:
###Computing the first matrix: the upper diagonal contains the mutual information for the obs;
###the lower diagonal is for the cnn-multi-site-multi-gaussian

mi.matrix <- mi.matrix.obs

idx.lower.tri <- which(lower.tri(mi.matrix[["JJAS"]]), arr.ind = T)

###replacing the lower triangle matrix
mi.matrix[["JJAS"]][idx.lower.tri] <- mi.matrix.list[[grep("CNN-MS-MG", preds)]][["JJAS"]][idx.lower.tri] 

l1 <- levelplot(mi.matrix$JJAS, ylab = "AEMET_13UTC_FWI13 (Obs)" ,
                xlab = "CNN-MG",
                scales = scales.list, main = "", col.regions = mi.colors(100),
               colorkey = list(space = "bottom", height = 1, width = 1)) 

In [ ]:
###Replacing with NAs those values with MI < 0.05 (below the threshold defined)

idx.na <- which(mi.matrix.obs$JJAS < 0.05, arr.ind = T)

i <- 1   
mi.matrix <- mi.matrix.list[[i]]
idx.lower.tri <- which(lower.tri(mi.matrix[["JJAS"]]), arr.ind = T)
mi.matrix[["JJAS"]][idx.lower.tri] <- mi.matrix.list[[i+1]][["JJAS"]][idx.lower.tri]
  
mi.matrix[["JJAS"]][idx.na] <- NA
  
l2 <- levelplot(mi.matrix[["JJAS"]]-mi.matrix.obs[["JJAS"]], ylab = sub("FWI-", "", preds[+1]),
                 xlab = sub("FWI-", "", preds[i+1]),scales = scales.list, main = "",
                 col.regions = rev(spec.colors(100)), at = seq(-0.15,0.15,0.01),
                 colorkey = list(space = "bottom", height = 1, width = 1)) 

pdf("./figures/MI_Matrices.pdf", width = 22.5, height = 11)

grid.arrange(l1, l2, nrow = 1, ncol = 2)

dev.off()

## 4.3 Model explainability <a class="anchor" id="4.3-bullet"></a>

Here, we present the aggregated saliency maps for the most representative predictor variables during FWI90 events across different climatic regions. The mean FWI saliency maps exhibit similar spatial distributions, with the main differences being in the intensity of feature significance and the comparable contributions from various variables. As a result, the FWI90 saliency maps are considered more suitable for analyzing the contributions of predictors to model construction, particularly in the context of the most impactful wildfire events.

Specifically, we focus on "saliency maps" (SM), which are spatial representations of feature importance in the model outputs. The relevance of each variable is assessed primarily by calculating the gradients of the output with respect to the input. These gradients are then back-propagated through the hidden layers of the network and visualized as saliency maps. Various techniques exist for computing this variable relevance; in this work, we use the Integrated Gradients (IG) method. For further details, please refer to the paper attached to this notebook.

In line with previous research, we post-process the "raw" output of the XAI technique by following these steps:
+ **`Step 1`**: We first compute the absolute value since the interest lies in the value regardless of its sign.
+ **`Step 2`**: To avoid gradient shattering, we filter out the values above a threshold of 1.5E-3.
+ **`Step 3`**: The percentage for each saliency map is individually calculated by dividing the value of each feature by the total sum of all features.
+ **`Step 4`**: We filter out the resultant gradients again with a threshold of 1.5E-3333 (0.15% relevance).
+ **`Step 5`**: Compute the percentage of each saliency map. Lastly, the saliency maps are aggregated by averaging the values over the training period, resulting in a collection of maps with the same dimensions as the input features, each representing a predictor variable.

In [ ]:
topology <- "cnn-multi-site-multi-gaussian"
name_sites <- y[["Metadata"]][["name"]]
x_scaled <- scaleGrid(x,type = "standardize")

##Loop over locations 
grid_intgra <- lapply(1:getShape(y, "loc"), FUN = function(ind_site) {
  station.id <- y[["Metadata"]][["station_id"]][ind_site]
  y_site <- subsetGrid(y, station.id = station.id) %>% redim(member = FALSE, loc = TRUE)
  
  name_site <- y[["Metadata"]][["name"]]ind_site]
  xCoord <- y[["xyCoords$"]][["x"]][ind_site]
  yCoord <- y[["xyCoords$"]][["y"]][ind_site]
  
  #Arrange input data
  x_scaled <- intersectGrid(x_scaled, y_site, type = "temporal")
  x_scaled[["Data"]] %<>% drop
  input <- if (topology == "cnn-multi-site" || topology == "cnn-multi-site-gaussian") {
    aperm(x_scaled[["Data"]], c(2,3,4,1))
  } else if (topology == "cnn-multi-site-multi-gaussian") {
    aperm(x_scaled[["Data"]], c(2,3,4,1))
  }
  
  
  #Computing integrated gradients
  batch <- 200
  samples <- dim(input)[1]
  init_batches <- seq(1, samples, batch)
  end_batches <- c(seq(batch, samples, batch), samples) %>% unique()
  saliency_array <- lapply(1:length(init_batches), FUN = function(ind_batch) {
    init_batch <- init_batches[ind_batch]
    end_batch <- end_batches[ind_batch]
    print(sprintf("Batch %i/%i", which(init_batch == init_batches), length(init_batches)))
    input_to_compute_intgra <- if (topology == "cnn-multi-site" || topology == "cnn-multi-site-gaussian" || topology == "cnn-multi-site-multi-gaussian") {
      input[init_batch:end_batch,,,]
    }
    get_integrated_gradients(input = input_to_compute_intgra, 
                             site = ind_site, 
                             model = model,
                             baseline = NULL, 
                             num_steps = 50) 
  }) %>% abind::abind(along = 1)
  

  #Free memory
  gc()
  
  
  ### Computing percentage and post-processing:
  ##Step 1: take the absolute value
  saliency_array %<>% abs()
  ##Step 2 and 3: filter the lowest values and compute saliency maps by dividing the value by the total sum of each feature
  filter <- 0.0015
  saliency_array[saliency_array < filter] <- 0
  for (i in 1:dim(saliency_array)[1]) {
    saliency_array[i,,,] <- saliency_array[i,,,] / sum(saliency_array[i,,,]) 
  }
  ##Step 4 and 5: Filter again and compute the percentage of each saliency map
  saliency_array[saliency_array < filter] <- 0
  for (i in 1:dim(saliency_array)[1]) {
    saliency_array[i,,,] <- saliency_array[i,,,] / sum(saliency_array[i,,,]) 
  }
  
  #Insert into a c4r object
  template <- x
  template[["Data"]] <- aperm(saliency_array, c(4,1,2,3))
  attr(template[["Data"]], "dimensions") <- c("var", "time", "lat", "lon")
  
  return(template)

}) %>% bindGrid(dimension = "member")

Finally, we plot the aggregated saliency maps for the most representative predictor variables during FWI90 events across various climatic regions. To achieve this, we load the `station_table.csv` file available in `utils` to categorize the stations by their corresponding climatic regions, thereby obtaining customized saliency maps for each distinct climatic region.

In [ ]:
regions <- c("COASM","ATL","CONTM")
figs.region <- list()
stations <- read.csv("./utils/station_table.csv")


cb <- brewer.pal(n = 8, "OrRd")
cb <- c("#FFFFFF", cb) %>% colorRampPalette()

In [ ]:
for(ind_region in 1:length(regions)) {
  region <- regions[ind_region]
  print(region)
  if (region == "COASM") {ind_station.ids <- which(stations[["climatic_region"]] == "COASM")}
  if (region == "ATL") {ind_station.ids <- which(stations[["climatic_region"]] == "ATL")}
  if (region == "CONTM") {ind_station.ids <- which(stations[["climatic_region"]] == "CONTM")}

  station.ids <- y[["Metadata"]][["station_id"]][ind_station.ids]
  y.st <- subsetGrid(y, station.id = station.ids)
  coords_region <- y.st[["xyCoords"]]

  
  ####fwi90 saliency maps####
  st.saliency.list <- lapply(station.ids, FUN = function(id){
    st <- subsetGrid(y.st, station.id = id)
    idx.fwi90 <- which(st[["Data"]] >= quantile(st[["Data"]], .9))
    grid_fwi90 <- subsetDimension(grid_intgra, dimension = "time", indices = idx.fwi90)
    grid_fwi90 <- subsetGrid(grid_fwi90, members = which(id == y[["Metadata"]][["station_id)"]]) %>% redim(member = FALSE) %>%
      aggregateGrid(aggr.mem = list(FUN = "mean", na.rm = TRUE))  %>%
      climatology() %>% redim(drop = T) %>% gridArithmetics(100, operator = "*") %>% redim(drop = T)
  })
  grid_fwi90 <- bindGrid(st.saliency.list, dimension = "member", skip.temporal.check = T) %>% climatology(by.member = F) %>% redim(drop = T)
  
 
  
  sp.layout.args <- list(list(sp::SpatialPoints(coords_region),
                              first = FALSE,
                              col = "black",
                              pch = 16))
  ##plotting....##
 
  max.value <- max(grid_fwi90[["Data"]])
  print(paste0("hus: ", max(grid_fwi90[["Data"]][1,,])))
  print(paste0("tas: ", max(grid_fwi90[["Data"]][3,,])))
  print(paste0("va: ", max(grid_fwi90[["Data"]][5,,])))
  
  if(region == "COASM"){
    at <- seq(0,1,.05)
  }else if(region == "ATL"){
    at <- seq(0,2,.1)
  }else if(region == "CONTM"){
    at <- seq(0,1.4,.07)
  }
  
  figs_fwi90 <- lapply(getVarNames(grid_fwi90), FUN = function(var) {
    idx.var <- which(getVarNames(grid_fwi90) == var)
    
    if(var == "hus850"){
      #at and cb values depending on the variable
      cb.new <- brewer.pal(n = 8, "Blues")
      cb.new <- c("#FFFFFF", cb.new) %>% colorRampPalette()
      at.new <- seq(0,0.3,0.015)
      spatialPlot(subsetGrid(grid_fwi90, var = var),
                  backdrop.theme = "coastline",
                  col.regions = cb.new,
                  colorkey = list(draw = T, space = "bottom", width = .95,labels = list(cex = 1.4)),
                  ylab = var,
                  set.min = 0,
                  set.max = at.new[length(at.new)],
                  at = at.new,
                  sp.layout = sp.layout.args,
                  cex.labels = 1.4)
      
    }else if(var == "va850"){
    
      cb.new <- brewer.pal(n = 8, "YlGn")
      cb.new <- c("#FFFFFF", cb.new) %>% colorRampPalette()
      at.new <- seq(0,0.5,0.025)
      spatialPlot(subsetGrid(grid_fwi90, var = var),
                  backdrop.theme = "coastline",
                  col.regions = cb.new,
                  colorkey = list(draw = T, space = "bottom", width = .95,labels = list(cex = 1.4)),
                  ylab = var,
                  set.min = 0,
                  set.max = at.new[length(at.new)],
                  at = at.new,
                  sp.layout = sp.layout.args,
                  cex.labels = 1.4)
      
    }else{
      
      spatialPlot(subsetGrid(grid_fwi90, var = var),
                  backdrop.theme = "coastline",
                  col.regions = cb,
                  colorkey = list(draw = T, space = "bottom", width = .95,labels = list(cex = 1.4)),
                  ylab = var,
                  set.min = 0,
                  set.max = at[length(at)],
                  at = at,
                  sp.layout = sp.layout.args,
                  cex.labels = 1.4)
      
    }
  })
  
  
  
  figs.region[[length(figs.region)+1]] <- grid.arrange(grobs = c(list(figs_fwi90[[1]], 
                                                                      figs_fwi90[[3]], 
                                                                      figs_fwi90[[5]])),
                                                       ncol = 3, as.table = FALSE, top = region)
  
}

pdf("./figures/explainability.pdf", width = 15, height = 20)
grid.arrange(figs.region[[1]], figs.region[[2]], figs.region[[3]])
dev.off()
